### In order to debug the torch transfer of weights, we're gonna side by side compare the forward passes of the two models... :(

Start with loading keras generator network

In [1]:
import sys
# point path to genesis repo
sys.path.insert(
    0,
    '/gpfs/commons/home/tchen/al_project/genesis/analysis/splicing'
)
from definitions.generator.splirent_deconv_conv_generator_concat import load_generator_network
from genesis.generator import build_generator, st_sampled_softmax, st_hardmax_softmax
from pathlib import Path
from keras.models import load_model
from keras import backend as K
import numpy as np
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Concatenate, Reshape, Softmax, Conv2DTranspose
from keras import Model

Using TensorFlow backend.


In [2]:
model_name = 'genesis_splicing_cnn_target_isoform_00_pwm_and_multisample_hek_only_random_regions_50_epochs_harderentropy_generator.h5'
model_save_dir = '/gpfs/commons/groups/knowles_lab/ting/DEN_splicing_pretrained_models/'

full_path = model_save_dir + model_name
generator_model = load_model(filepath=str(full_path), custom_objects={'K': K, 'st_sampled_softmax': st_sampled_softmax, 'st_hardmax_softmax': st_hardmax_softmax}, compile=False)

2024-03-12 12:55:30.572975: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2799950000 Hz
2024-03-12 12:55:30.573999: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55654fc66e80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-03-12 12:55:30.574025: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Instructions for updating:
Use `tf.random.categorical` instead.



Now we load the torch model with the previously saved weights

In [3]:
import sys
# point path to our repo
sys.path.insert(
    0,
    '/gpfs/commons/home/tchen/al_project/active-learning-cnns-gps/src'
)

import numpy as np
from models.den import Generator
import torch
from torch import nn
import torch.nn.functional as F

In [4]:
save_path = '/gpfs/commons/groups/knowles_lab/ting/DEN_splicing_generator_weights/'
save_name = 'target_isoform_00.npy'

target_isoform_net_weights = np.load(save_path+save_name, allow_pickle=True)

# list of numpy arrays
weights = target_isoform_net_weights.tolist()
device = torch.device('cpu')

# need to extract embedding template and mask from pretrained network
pretrained_embedding_template = torch.tensor(weights[-2]).to(device)
pretrained_embedding_mask = torch.tensor(weights[-1]).to(device)

n_samples = 10
n_classes = 1
seq_length = 109
latent_dim = 100
batch_size = 32

torch_generator = Generator(embedding_template=pretrained_embedding_template,
                                   embedding_mask=pretrained_embedding_mask,
                                   device=device,
                                   latent_dim=latent_dim,
                                   batch_size=batch_size,
                                   seq_length=seq_length,
                                   n_classes=n_classes,
                                   n_samples=n_samples)


In [5]:

running_keras_weight_idx = 0

for i, layer in enumerate(torch_generator.generator_network.generator_network):
    if isinstance(layer, nn.Linear):
        # transfer linear layer weights and biases
        assert tuple(layer.weight.data.shape) == weights[running_keras_weight_idx].T.shape
        layer.weight.data = torch.from_numpy(weights[running_keras_weight_idx].T)
        running_keras_weight_idx += 1

        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.ConvTranspose2d):
        # transfer convtranspose2d weights and biases
        assert tuple(layer.weight.shape) == np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]).shape
        layer.weight.data = torch.from_numpy(np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]))
        running_keras_weight_idx += 1
        
        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.Conv2d):
        # transfer conv2d weights and biases
        assert tuple(layer.weight.data.shape) == np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]).shape
        layer.weight.data = torch.from_numpy(np.transpose(weights[running_keras_weight_idx], axes=[3, 2, 0, 1]))
        running_keras_weight_idx += 1
        
        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
    elif isinstance(layer, nn.BatchNorm2d):
        # transfer batch norm gamma, beta, running mean, running variance
        # order from keras model should be gamma, beta, moving mean, moving variance
        assert tuple(layer.weight.data.shape) == weights[running_keras_weight_idx].shape # this might need a transpose
        layer.weight.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        assert layer.bias.data.shape == weights[running_keras_weight_idx].shape
        layer.bias.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        assert layer.running_mean.data.shape == weights[running_keras_weight_idx].shape
        layer.running_mean.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1
        
        assert layer.running_var.data.shape == weights[running_keras_weight_idx].shape
        layer.running_var.data = torch.from_numpy(weights[running_keras_weight_idx])
        running_keras_weight_idx += 1

In [6]:
torch_generator.to(device)
torch_generator.eval()

Generator(
  (generator_network): GeneratorNetwork(
    (generator_network): ModuleList(
      (0): Linear(in_features=101, out_features=3456, bias=True)
      (1): ConvTranspose2d(384, 256, kernel_size=(7, 1), stride=(2, 1))
      (2): BatchNorm2d(256, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (3): ConvTranspose2d(256, 192, kernel_size=(8, 1), stride=(2, 1))
      (4): BatchNorm2d(192, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (5): ConvTranspose2d(192, 128, kernel_size=(7, 1), stride=(2, 1))
      (6): BatchNorm2d(128, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (7): Conv2d(128, 128, kernel_size=(8, 1), stride=(1, 1), padding=same)
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (9): Conv2d(128, 64, kernel_size=(8, 1), stride=(1, 1), padding=same)
      (10): BatchNorm2d(64, eps=1e-05, momentum=0.99, affine=True, track_running_stats=True)
      (11): 

Okay now we'll start with the same input and layer by layer compare the outputs of the keras model with the outputs of the torch model

In [7]:
# generator forward loop (up until generator network)

# Seed class input for all dense/embedding layers
sequence_class = torch.empty(batch_size).to(device)

# TODO: Super jank, refactor this...
sequence_class.uniform_(-0.499, 1-0.5001)
sequence_class = torch.round(sequence_class).int()
# sequence_class = torch.ones(self.batch_size, dtype=torch.int32).to(self.device)

# Get generated policy pwm logits (non-masked)
# latent_input_1 = torch.empty((batch_size, latent_dim)).to(device)
# latent_input_2 = torch.empty((batch_size, latent_dim)).to(device)
latent_input_1 = torch.ones((batch_size, latent_dim)).to(device)
latent_input_2 = torch.ones((batch_size, latent_dim)).to(device)

# latent_input_1.uniform_(-1.0, 1.0)
# latent_input_2.uniform_(-1.0, 1.0)

sequence_class_onehots = torch.eye(1).to(device)

# check here for bugs
class_embedding = sequence_class_onehots.index_select(0, index=sequence_class)    # tf.gather equivalent
# class_embedding = torch.ones((self.batch_size, 1), dtype=torch.int32).to(self.device)

torch_seed_input_1 = torch.cat([latent_input_1, class_embedding], dim=-1)
torch_seed_input_2 = torch.cat([latent_input_2, class_embedding], dim=-1)

In [8]:
# repeat same for keras model
sequence_class_input = Input(tensor=K.ones((batch_size, 1)), dtype='int32', name='sequence_class_seed')
sequence_class = Lambda(lambda inp: K.cast(K.round(inp * K.random_uniform((batch_size, 1), minval=-0.4999, maxval=n_classes-0.5001)), dtype='int32'), name='lambda_rand_sequence_class')(sequence_class_input)

sequence_class_onehots = np.eye(n_classes)

#Generator network parameters
latent_size = 100

#Generator inputs
latent_input_1 = Input(tensor=K.ones((batch_size, latent_size)), name='noise_input_1')
latent_input_2 = Input(tensor=K.ones((batch_size, latent_size)), name='noise_input_2')
# latent_input_1_out = Lambda(lambda inp: inp * K.random_uniform((batch_size, latent_size), minval=-1.0, maxval=1.0), name='lambda_rand_input_1')(latent_input_1)
# latent_input_2_out = Lambda(lambda inp: inp * K.random_uniform((batch_size, latent_size), minval=-1.0, maxval=1.0), name='lambda_rand_input_2')(latent_input_2)

class_embedding = Lambda(lambda x: K.gather(K.constant(sequence_class_onehots), K.cast(x[:, 0], dtype='int32')))(sequence_class)

keras_seed_input_1 = Concatenate(axis=-1)([latent_input_1, class_embedding])
keras_seed_input_2 = Concatenate(axis=-1)([latent_input_2, class_embedding])

In [9]:
func = K.function([sequence_class_input, latent_input_1, latent_input_2], [keras_seed_input_1, keras_seed_input_2])
keras_sequence_class = func([K.ones((batch_size, 1)), K.ones((batch_size, latent_size)), K.ones((batch_size, latent_size))])

All good so far from here, let's start testing the generator networks layer by layer

In [10]:
torch_dense_output = F.relu(torch_generator.generator_network.generator_network[0](torch_seed_input_1))

In [11]:
# copy dense layer weights over
layer_input = Input(shape=(latent_dim+1,))
copy_dense_1 = Dense(9 * 384, activation='relu', kernel_initializer='glorot_uniform', name='policy_dense_1')
out = copy_dense_1(layer_input)
model = Model(inputs=layer_input, outputs=out)
copy_dense_1.set_weights(generator_model.get_layer('policy_dense_1').get_weights())

In [12]:
keras_dense_output = model.predict(keras_sequence_class[0])

In [13]:
np.testing.assert_allclose(torch_dense_output.detach().numpy(), keras_dense_output, rtol=1e-03) 

First dense layer passed

In [43]:
torch_generator.generator_network.generator_network[1]

ConvTranspose2d(384, 256, kernel_size=(7, 1), stride=(2, 1))

In [14]:
torch_deconv_0_output = torch_generator.generator_network.generator_network[1](torch_dense_output.view(batch_size, 384, 9, 1))

In [35]:
torch_deconv_0_output

tensor([[[[  6.5826],
          [ -1.9226],
          [  1.5328],
          ...,
          [-10.2842],
          [ -4.4818],
          [ -4.3741]],

         [[ -5.2752],
          [  1.5402],
          [ -9.7774],
          ...,
          [-17.4850],
          [-13.4211],
          [ -7.0050]],

         [[ -2.1654],
          [  8.2707],
          [-13.3770],
          ...,
          [-12.9350],
          [  4.1130],
          [ -0.6644]],

         ...,

         [[ -7.1633],
          [  4.9998],
          [ -4.7229],
          ...,
          [ -0.5790],
          [ -3.8544],
          [  0.5504]],

         [[ -0.4081],
          [ -3.9280],
          [  3.7245],
          ...,
          [-30.1781],
          [ -3.3762],
          [-19.1783]],

         [[-11.1019],
          [  6.2191],
          [ -5.6924],
          ...,
          [ 19.4362],
          [ -2.9812],
          [ 16.9553]]],


        [[[  6.5826],
          [ -1.9226],
          [  1.5328],
          ...,
        

In [39]:
layer_input = Input(shape=(9*384,))
policy_dense_1_reshape = Reshape((9, 1, 384))
policy_deconv_0 = Conv2DTranspose(256, (7, 1), strides=(2, 1), padding='valid', activation='linear', kernel_initializer='glorot_normal', name='policy_deconv_0', input_shape=(9, 1, 384), data_format='channels_last')
out_reshape = policy_dense_1_reshape(layer_input)
out = policy_deconv_0(out_reshape)
model = Model(inputs=layer_input, outputs=[out, out_reshape])
policy_deconv_0.set_weights(generator_model.get_layer('policy_deconv_0').get_weights())
keras_deconv_0_output = model.predict(keras_dense_output)

In [40]:
# check after reshape
np.testing.assert_allclose(keras_deconv_0_output[-1], torch_dense_output.view(batch_size, 9, 1, 384).detach().numpy(), rtol=1e-3)

In [41]:
torch_deconv_0_output.detach().numpy().shape == keras_deconv_0_output[0].transpose([0, 3, 1, 2]).shape

True

In [42]:
np.testing.assert_allclose(torch_deconv_0_output.detach().numpy(), keras_deconv_0_output[0].transpose([0, 3, 1, 2]), rtol=1e-3) 

AssertionError: 
Not equal to tolerance rtol=0.001, atol=0

Mismatched elements: 188192 / 188416 (99.9%)
Max absolute difference: 57.362293
Max relative difference: 850.39166
 x: array([[[[  6.582626],
         [ -1.922581],
         [  1.532797],...
 y: array([[[[ 2.139414e+00],
         [-3.674752e-03],
         [ 5.357710e+00],...

Looks like the error was in the conv2dTranspose...

In [20]:
deconv_0 = nn.ConvTranspose2d(in_channels=384, out_channels=256, kernel_size=(7, 1), stride=(2, 1), bias=True)

In [21]:
assert deconv_0.weight.shape == np.transpose(generator_model.get_layer('policy_deconv_0').get_weights()[0], axes=[3, 2, 0, 1]).shape
deconv_0.weight.data = torch.from_numpy(np.transpose(generator_model.get_layer('policy_deconv_0').get_weights()[0], axes=[3, 2, 0, 1]))
deconv_0.bias.data = torch.from_numpy(generator_model.get_layer('policy_deconv_0').get_weights()[1])

In [41]:
generator_model.get_layer('policy_deconv_0').get_weights()[0].shape # kernel height, width, out channels, in channels

(7, 1, 256, 384)

In [ ]:

generator_model.get_layer('policy_deconv_0').compute_output_shape((32, 9, 1, 384))

In [40]:
deconv_0.weight.shape # in channels, out channels, kernel height, kernel width

torch.Size([384, 256, 7, 1])

In [46]:
# check weights are equal
np.testing.assert_allclose(torch_generator.generator_network.generator_network[1].weight.detach().numpy(), policy_deconv_0.get_weights()[0].transpose([3, 2, 0, 1])) 
np.testing.assert_allclose(torch_generator.generator_network.generator_network[1].bias.detach().numpy(), policy_deconv_0.get_weights()[1]) 

In [44]:
# test with ones as input

keras_test_ones_output = model.predict(np.ones((32, 9*384)))
torch_test_ones_output = torch_generator.generator_network.generator_network[1](torch.ones((32, 384, 9, 1)))
np.testing.assert_allclose(torch_test_ones_output.detach().numpy().transpose(0, 2, 3, 1), keras_test_ones_output[0], rtol=1e-03) 

In [45]:
# test with ones as input

keras_test_ones_output = model.predict(np.ones((32, 9*384)))
torch_test_ones_output = deconv_0(torch.ones((32, 384, 9, 1)))
np.testing.assert_allclose(torch_test_ones_output.detach().numpy().transpose(0, 2, 3, 1), keras_test_ones_output[0], rtol=1e-03) 

In [ ]:
torch_generator.generator_network.generator_network[1]

In [25]:
# test with random normal as input
rand_input = np.random.normal(size=(32, 9 * 384))
keras_test_rand_output = model.predict(rand_input)
torch_test_rand_output = torch_generator.generator_network.generator_network[1](torch.from_numpy(rand_input).view((32, 384, 9, 1)).float())
np.testing.assert_allclose(torch_test_rand_output.detach().numpy().transpose(0, 2, 3, 1), keras_test_rand_output[0], rtol=1e-05) 

AssertionError: 
Not equal to tolerance rtol=1e-05, atol=0

Mismatched elements: 188416 / 188416 (100%)
Max absolute difference: 37.82418
Max relative difference: 101483.47
 x: array([[[[  5.396858,   3.702867,   3.895614, ...,   2.196572,
            1.287122,   2.091106]],
...
 y: array([[[[ -5.685665,  -1.733513,  -6.924087, ...,   1.014218,
            5.29361 ,  -0.489688]],
...

In [ ]:
# test with random normal as input
rand_input = np.random.normal(size=(32, 9 * 384))
keras_test_rand_output = model.predict(rand_input)
torch_test_rand_output = deconv_0(torch.from_numpy(rand_input.reshape(32, 384, 9, 1)).float())
np.testing.assert_allclose(torch_test_rand_output.detach().permute(0, 2, 3, 1).numpy(), keras_test_rand_output[0], rtol=1e-05) 

In [101]:
X = np.array([[[1.0, 1.0, 1.0], [2.0, 1.0, 1.0]], [[2.0, 1.0, 1.0], [1.0, 1.0, 0.0]],  [[2.0, 2.0, 2.0], [1.0, 1.0, 1.0]]])
X = X.reshape(1, 3, 2, 3)
weights = [np.asarray([[[[1, 0, 1], [1, 0, 1]]], [[[1, 0, 1], [1, 0, 1]]],  [[[1, 0, 1], [1, 0, 1]]]]), np.asarray([2, 2])]


In [106]:
X

array([[[[1., 1., 1.],
         [2., 1., 1.]],

        [[2., 1., 1.],
         [1., 1., 0.]],

        [[2., 2., 2.],
         [1., 1., 1.]]]])

In [130]:
weights[0]

array([[[[1, 0, 1],
         [1, 0, 1]]],


       [[[1, 0, 1],
         [1, 0, 1]]],


       [[[1, 0, 1],
         [1, 0, 1]]]])

In [136]:
weights[0].shape

(3, 1, 2, 3)

In [138]:
weights[0].transpose(0, 2, 3, 1).shape

(3, 2, 3, 1)

In [172]:
torch_dummy = nn.ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=(3, 1), stride=(2, 1), bias=True)
print(torch_dummy.weight.data.shape, torch.from_numpy(weights[0].transpose(3, 2, 0, 1)).float().shape)
assert torch_dummy.weight.data.shape ==  torch.from_numpy(weights[0].transpose(3, 2, 0, 1)).float().shape
torch_dummy.weight.data = torch.from_numpy(weights[0].transpose(3, 2, 0, 1)).float()
torch_dummy.bias.data = torch.from_numpy(weights[1]).float()

torch.Size([3, 2, 3, 1]) torch.Size([3, 2, 3, 1])


In [175]:
torch_dummy(torch.from_numpy(X).float())[0][0]

tensor([[5., 5., 5.],
        [5., 5., 5.],
        [8., 7., 7.],
        [5., 4., 4.],
        [5., 4., 4.]], grad_fn=<SelectBackward0>)

In [31]:
import keras

In [141]:
layer_input = Input(shape=(3, 2, 3,))
reshape = Reshape((2, 3, 3))
conv = keras.layers.Conv2DTranspose(2, (3, 1), strides=(2, 1), padding='valid', activation='linear', kernel_initializer='glorot_normal')
out = conv(reshape(layer_input))

model_Conv2D = Model(layer_input, out)
conv.set_weights(weights)
yhat = model_Conv2D.predict(X)

In [171]:
yhat.reshape(5, 3, 2)[:, :, 0]

array([[4., 5., 5.],
       [4., 5., 5.],
       [5., 9., 7.],
       [3., 6., 4.],
       [3., 6., 4.]], dtype=float32)

In [143]:
np.testing.assert_allclose(yhat.reshape(5, 3, 2), torch_dummy(torch.from_numpy(X).float()).permute(0, 2, 3, 1).squeeze().detach().numpy())

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 26 / 30 (86.7%)
Max absolute difference: 6.
Max relative difference: 2.
 x: array([[[4., 4.],
        [5., 5.],
        [5., 5.]],...
 y: array([[[ 7.,  7.],
        [ 6.,  6.],
        [ 6.,  6.]],...